# An example of using the python eTaSL driver in a python notebook (2)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML

#%load_ext autoreload
#%autoreload 2

In [ ]:
# change the width of the notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Simulating the eTaSL controller using the high-level eTaSL driver

In [ ]:
a=['a','b']
b=['c','b','d','a']
def force_order( ord, all):
    
    return

In [ ]:


def force_order( lst, ord): 
    s_ord = {e:True for e in ord}
    result = [e for e in ord]
    result.extend( [e for e in lst if not e in s_ord] )
    return result

a=['a','b']
b=['d','b','e','a']
print(  force_order(b, a))

In [ ]:
from etasl_py.etasl import  etasl,to_rad,to_deg,integrate,array_to_dict,dict_to_array
class etasl_simulator:
    def __init__(self, 
                    nWSR                  = 300,
                    cputime               = 1000,
                    regularization_factor = 1E-4
                ):
        self.nWSR=nWSR
        self.cputime=cputime
        self.regularization_factor = regularization_factor
        self.etasl  = etasl(self.nWSR,self.cputime,self.regularization_factor)
        
    def initialize(self, initial_jpos, pos_lbl, max_time=3,time_step=0.05,convergence_crit=1E-4):
        """def initialize(self,initialval, initial_lbl, max_time, time_step, convergence_crit):
        
        Initializes the controller and performs an optimization to compute an optimal start value 
        for the feature variables.

        Performs the following tasks in this order:
           1) prepares the solver for the initialization problem
           2) initializes the state (robot/feature names, values and velocities)
           3) sets the initial value for robot/feature variables 
           4) performs an optimization to compute an optimal start value 
              for the feature variables (only taking into account the constraints with priority==0)
           5) prepares the solver for the exuction problem and solves one step
              (such that next steps are all hot-starts)
        
        Args:
           initialval:           value of robot- and feature variables you 
                                 want to initialize (before initialization optimization)
                                 (if not specified, the initial value of the specification
                                  is used.)
           initial_lbl:          labels belonging the initial value array.
           initialization_time:  max. (virtual) time to use for initialization
           sample_time:          (virtual) sample time to use for the initialization
           convergence_crit:     convergence criterion used to stop the initialization early.
       Warning:
           - initializes time to 0, you can overwrite this in the initialval map.
           - robot variables remain identical as specified after this method call.
           - feature variables can be changed after this method call (if they are involved
             in the initialization optimization). 
     
        Returns:
           convergedval:         variable to return the converged values for joint and 
                                 feature variables. 
 
            TaskSpecificationException, NumericalException, LogicalException

        """
        self.initial_jpos = array_to_dict( initial_jpos, pos_lbl )
        self.preferedJointOrder = pos_lbl
        print(self.initial_jpos)
        self.initial_jpos = self.etasl.initialize(self.initial_jpos,max_time=3,time_step=0.05,convergence_crit=1E-4)
        
    def __str__(self):
        return """etasl_simulator(
                     WSR : {}
                     cputime : {}
                     regularization_factor : {}
                     # input callbacks registered : {}
                     # output callbacks registered : {}
                  )""".format(
                            self.nWSR, 
                            self.cputime, 
                            self.regularization_factor,
                            len(self.input_cb),
                            len(self.output_cb)
                    ) 
    def getInputNames(self):
        return self.etasl.getInputNames()
    def getOutputNames(self):
        return self.etasl.getOutputNames()
    def getVariables(self,flag):
        return self.etasl.getVariables(flag)
    def nrOfBoxConstraints(self):
        return self.etasl.nrOfBoxConstraints()
    def nrOfScalarConstraints(self):
        return self.etasl.nrOfScalarConstraints()
    def readTaskSpecificationFile(self,name):
        return self.etasl.readTaskSpecificationFile(name)
    def readTaskSpecificationString(self,estr):
        return self.etasl.readTaskSpecificationString(estr)
    def printContext(self):
        return self.etasl.printContext()
    # copy doc from underlying object for a given set of methods:
    __init__.__doc__ = etasl.__init__.__doc__
    readTaskSpecificationFile.__doc__=etasl.readTaskSpecificationFile.__doc__
    getInputNames.__doc__ = etasl.getInputNames.__doc__
    getOutputNames.__doc__ = etasl.getOutputNames.__doc__
    nrOfBoxConstraints.__doc__=etasl.nrOfBoxConstraints.__doc__
    nrOfScalarConstraints.__doc__ = etasl.nrOfScalarConstraints.__doc__
    readTaskSpecificationString.__doc__ = etasl.readTaskSpecificationString.__doc__
    printContext.__doc__ = etasl.printContext.__doc__
    
    #def add_input_cb(self, f ):
    #    self.input_cb.append(f);
    #def add_output_cb(self, f ):
    #    self.output_cb.append(f);    
        
    def setInputTable(self, inp_lbl, inp, inpvel=None):
        '''
        sets input for the simulation,
        
        input:
            inplbl
            inptable:  
            veltable:  OPTIONAL: 
        '''
        self.INP_LBL = inp_lbl
        self.INP = inp
        self.INPVEL = inpvel
        
    def simulate(self, N, dt):
        """
        obj.simulate(N,dt)
        
        input parameters:
            N:  number of simulation steps
            dt: sample period
        """
        self.TIME   = np.arange(0,N)*dt
        self.POS_LBL         = force_order( [v for v in self.getVariables(3)], self.preferedJointOrder );
        #LBL.append('time')
        self.POS    = np.zeros( ( N , len(self.POS_LBL) ) )        
        self.VEL    = np.zeros( ( N , len(self.POS_LBL) ) )
        self.OUTP_LBL        = self.etasl.getOutputNames()
        self.OUTP   = np.zeros((len(self.TIME),len(self.OUTP_LBL)))
        pos         = {k:v[0] for k,v in self.getVariables(3).iteritems()};
        for k,v in self.initial_jpos.iteritems():
            pos[k]=v
        pos["time"] = self.TIME[0] 
        i =0
        while True:
            self.etasl.setInput(array_to_dict(self.INP[i,:],self.INP_LBL))
            if not self.INPVEL is None:
                self.etasl.setInputVelocity(array_to_dict(self.INPVEL[i,:],self.INP_LBL)) 
            self.etasl.setJointPos(pos)
            self.etasl.solve()
            vel = self.etasl.getJointVel(3)
            pos = integrate(pos, vel, dt)
            self.OUTP[i,:] = dict_to_array(self.etasl.getOutput(), self.OUTP_LBL)
            self.POS[i,:] = dict_to_array(pos, self.POS_LBL)
            self.VEL[i,:] = dict_to_array(vel, self.POS_LBL)
            i    = i + 1
            if i >= len(self.TIME):
                break
    


In [ ]:
e = etasl_simulator(regularization_factor= 0.00001)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")

pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint', 'f1']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,0,0,0,0])



N=4000
dt=0.005
time = np.arange(0,N)*dt
inp_lbl=['tgt_x','tgt_y','tgt_z']
inp=np.zeros((N, len(inp_lbl)))
inp[:,0] = np.sin(time)*0.15 + 0.7
inp[:,1] = time*0
inp[:,2] = time*0 + 0.3
inpvel = np.zeros((N, len(inp_lbl)))
inpvel[:,0] = np.cos(time)*0.15
inpvel[:,1] = time*0
inpvel[:,2] = time*0

e.setInputTable(inp_lbl,inp,inpvel)
e.initialize(initial_jpos, pos_lbl)
e.simulate(N=N,dt=dt)

In [ ]:
print(e.initial_jpos)
pd.DataFrame(e.POS[1:3,:])


In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

TOOLTIPS = [
    ("(time,value)", "($x, $y)")
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

print("joint positions : ")
print(e.POS_LBL)
p=[]
for i in range(0,e.POS.shape[1]):
    f = figure(x_axis_label='time [s]', 
            y_axis_label= e.POS_LBL[i],     
            tooltips=TOOLTIPS,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset")
    if i==0:
        fx=f.x_range
    else:
        f.x_range=fx
    f.line(e.TIME, e.POS[:,i], line_width=2)
    p.append(f)

pg = gridplot( [[ p[0], p[1], p[2],p[3] ],[ p[4], p[5], p[6], None]], 
            plot_width=380, plot_height=250,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset" ) 
show(pg)

print("joint velocities : ")
print(e.POS_LBL)
p=[]
for i in range(0,e.POS.shape[1]):
    f = figure(x_axis_label='time [s]', 
            y_axis_label= "velocity of "+e.POS_LBL[i],    
            tooltips=TOOLTIPS,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset")
    if i==0:
        fx=f.x_range
    else:
        f.x_range=fx
    f.line(e.TIME, e.VEL[:,i], line_width=2)
    p.append(f)

pg = gridplot( [[ p[0], p[1], p[2],p[3] ],[ p[4], p[5], p[6], None]], 
            plot_width=380, plot_height=250,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset" ) 
show(pg)

print("error:")

pw=500
ph=400
# output to static HTML file

p1 = figure(x_axis_label='time', 
            y_axis_label= "error x",
            plot_width=pw,
            plot_height=ph)
p1.line(e.TIME, e.OUTP[:,0] - e.INP[:,0], line_width=2)

p2 = figure(x_axis_label='time', 
            y_axis_label= "error y",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p2.line(e.TIME, e.OUTP[:,1] - e.INP[:,1], line_width=2)

p3 = figure(x_axis_label='time', 
            y_axis_label= "error z",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p3.line(e.TIME, e.OUTP[:,2] - e.INP[:,2], line_width=2)

pg = gridplot( [ [p1,p2, p3 ] ])    
show(pg)

In [ ]:




def cb_table_input(df):
    '''returns a callback function that uses all columns of the given data 
       frame to fill in input values for eTaSL'''
    def cb( e, seqnr,time):
        '''callback that sets the input based on a table and the given seqnr'''
        e.setInput(df.iloc[seqnr])
        return
            
    return cb

def cb_table_output(df):
    def cb(e, seqnr,time):
        '''callback that stores output in a a table at the given seqnr'''
        row=e.getOutput()
        row['TimeStamp'] = time
        df.iloc[seqnr] = e.row
        return
    return cb
        

# Using matplotlib

In [ ]:
plotw=24
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.POS.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.POS[:,i]*180.0/np.pi)
    plt.ylabel(e.POS_LBL[i] +" [deg]")
    plt.xlabel('time [s]')
    
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.VEL.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.VEL[:,i])
    plt.ylabel(e.POS_LBL[i] +" [rad/s]")
    plt.xlabel('time [s]')

w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.INP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.INP[:,i])
    plt.ylabel(e.INP_LBL[i])
    plt.xlabel('time [s]')
    
w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.OUTP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.OUTP[:,i])
    plt.ylabel(e.OUTP_LBL[i])
    plt.xlabel('time [s]')    
    
    
plt.figure(figsize=(plotw,4))
plt.subplot(1,3,1)
plt.plot(e.TIME, e.OUTP[:,0] - e.INP[:,0])
plt.xlabel('time [s]')
plt.ylabel('error x')

plt.subplot(1,3,2)
plt.plot(e.TIME, e.OUTP[:,1] - e.INP[:,1])
plt.xlabel('time [s]')
plt.ylabel('error y')

plt.subplot(1,3,3)
plt.plot(e.TIME, e.OUTP[:,2] - e.INP[:,2])
plt.xlabel('time [s]')
plt.ylabel('error z')